In [67]:
#import packages
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine

def fetch_csv_from_github(url):
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful

    # Convert the raw data into a pandas DataFrame (handling tab separation)
    data = pd.read_csv(StringIO(response.text), delimiter='\t', header=None)

    # Return cleaned data
    return data

# URL of the raw CSV file on GitHub
salesorderdetail_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/SalesOrderDetail.csv"
salesterritory_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/SalesTerritory.csv"
purchaseorderdetail_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/PurchaseOrderDetail.csv"
shipmethod_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/ShipMethod.csv"
employee_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/Employee.csv"
POheader_csv = "https://raw.githubusercontent.com/dnguyen-1/AdventureWorks507/main/data/PurchaseOrderHeader.csv"


# Fetch Data
SalesOrderDetail = fetch_csv_from_github(salesorderdetail_csv)
SalesTerritory = fetch_csv_from_github(salesterritory_csv)
PurchaseOrderDetail = fetch_csv_from_github(purchaseorderdetail_csv)
ShipMethod = fetch_csv_from_github(shipmethod_csv)
Employee = fetch_csv_from_github(employee_csv)
PurchaseOrderHeader = fetch_csv_from_github(POheader_csv)

In [68]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql  # To handle MySQL connection

# MySQL connection string
server = 'team-shared-mysql.cjwa24wuisi8.us-east-1.rds.amazonaws.com'
database = 'AdventureSales'
username = 'Ads507'
password = 'Gabrielleo24'
connection_string = f'mysql+pymysql://{username}:{password}@{server}/{database}'

# Create the SQLAlchemy engine to connect to the database
engine = create_engine(connection_string)

# Function to load data into MySQL without replacing headers
def load_data_to_mysql():
    try:
        with engine.connect() as conn:
            # Remove existing data without dropping the table
            conn.execute(text("DELETE FROM PurchaseOrderDetail"))
            conn.execute(text("DELETE FROM SalesOrderDetail"))
            conn.execute(text("DELETE FROM SalesTerritory"))
            conn.execute(text("DELETE FROM ShipMethod"))
            conn.execute(text("DELETE FROM Employee"))
            conn.execute(text("DELETE FROM PurchaseOrderHeader"))
            conn.commit()  # Commit the transaction

        # Explicitly set DataFrame column names to match the SQL table schema
        PurchaseOrderDetail.columns = [
            'PurchaseOrderID', 'PurchaseOrderDetailID', 'DueDate', 'OrderQty',
            'ProductID', 'UnitPrice', 'LineTotal', 'ReceivedQty', 'RejectQty',
            'StockedQty', 'ModifiedDate'
        ]

        SalesOrderDetail.columns = [
            'SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
            'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice',
            'UnitDiscount', 'LineTotal', 'rowguid', 'ModifiedDate'
        ]

        SalesTerritory.columns = [
            'TerritoryID', 'Name', 'CountryRegionCode', 'Geo_Group', 'SalesYTD',
            'SalesLastYear', 'CostYTD', 'CostLastYear', 'rowguid', 'ModifiedDate'
        ]

        ShipMethod.columns = [
            'ShipMethodID', 'Name', 'ShipBase', 'ShipRate', 'rowguid', 'ModifiedDate'
        ]

        Employee.columns = [
            'BusinessEntityID', 'NationalIDNumber', 'LoginID', 'OrganizationNode',
            'OrganizationLevel', 'JobTitle', 'BirthDate', 'MaritalStatus', 'Gender',
            'HireDate', 'SalariedFlag', 'VacationHours', 'SickLeaveHours',
            'CurrentFlag', 'rowguid', 'ModifiedDate'
        ]

        PurchaseOrderHeader.columns = [
            'PurchaseOrderID', 'RevisionNumber', 'order_status', 'EmployeeID', 
            'VendorID', 'ShipMethodID', 'OrderDate', 'ShipDate', 'SubTotal', 
            'TaxAmt', 'Freight', 'TotalDue', 'ModifiedDate'
        ]


        # Append the new data (without replacing the table structure)
        PurchaseOrderDetail.to_sql('PurchaseOrderDetail', con=engine, if_exists='append', index=False)
        SalesOrderDetail.to_sql('SalesOrderDetail', con=engine, if_exists='append', index=False)
        SalesTerritory.to_sql('SalesTerritory', con=engine, if_exists='append', index=False)
        ShipMethod.to_sql('ShipMethod', con=engine, if_exists='append', index=False)
        Employee.to_sql('Employee', con=engine, if_exists='append', index=False)
        PurchaseOrderHeader.to_sql('PurchaseOrderHeader', con=engine, if_exists='append', index=False)

    except Exception as e:
        print(f"Error loading data: {e}")

# Call the function to load the data
load_data_to_mysql()


In [61]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql  # To handle MySQL connection

# MySQL connection string
server = 'team-shared-mysql.cjwa24wuisi8.us-east-1.rds.amazonaws.com'
database = 'AdventureSales'
username = 'Ads507'
password = 'Gabrielleo24'
connection_string = f'mysql+pymysql://{username}:{password}@{server}/{database}'

# Create the SQLAlchemy engine to connect to the database
engine = create_engine(connection_string)

# Function to load data into MySQL
def load_data_to_mysql():
    try:
        ##Customers.columns = ['CustomerID', 'PersonID', 'StoreID', 'TerritoryID', 'Accountnumber', 'rowguid', 'ModifiedDate']
        SalesOrderDetail.columns = ['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber', 'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice', 'UnitDiscount', 'LineTotal', 'rowguid', 'ModifiedDate']
        SalesTerritory.columns = ['TerritoryID', 'Name','CountryRegionCode', 'Geo_Group', 'SalesYTD', 'SalesLastYear', 'CostYTD', 'CostLastYear', 'rowguid', 'ModifiedDate']
        PurchaseOrderDetail.columns = ['PurchaseOrderID', 'PurchaseOrderDetailID', 'DueDate', 'OrderQty', 'ProductID', 'UnitPrice', 'LineTotal', 'ReceivedQty', 'RejectQty', 'StockedQty', 'ModifiedDate']
        ShipMethod.columns = ['ShipMethodID', 'Name', 'ShipBase', 'ShipRate', 'rowguid', 'ModifiedDate']
        Employee.columns = ['BusinessEntityID', 'NationalIDNumber', 'LoginID', 'OrganizationNode', 'OrganizationLevel', 'JobTitle', 'BirthDate', 'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag', 'VacationHours', 'SickLeaveHours', 'CurrentFlag', 'rowguid', 'ModifiedDate',]

        # Load the data into MySQL tables (replace existing tables)
        PurchaseOrderDetail.to_sql('PurchaseOrderDetail', con=engine, if_exists='replace', index=False)
        ##Customers.to_sql('Customers', con=engine, if_exists='replace', index=False)
        SalesOrderDetail.to_sql('SalesOrderDetail', con=engine, if_exists='replace', index=False)
        SalesTerritory.to_sql('SalesTerritory', con=engine, if_exists='replace', index=False)
        ShipMethod.to_sql('ShipMethod', con=engine, if_exists='replace', index=False)
        Employee.to_sql('Employee', con=engine, if_exists='replace', index=False)

        print("Data loaded successfully into MySQL.")
    
    except Exception as e:
        print(f"Error loading data: {e}")


In [39]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql  # To handle MySQL connection

# MySQL connection string
server = 'team-shared-mysql.cjwa24wuisi8.us-east-1.rds.amazonaws.com'
database = 'AdventureSales'
username = 'Ads507'
password = 'Gabrielleo24'
connection_string = f'mysql+pymysql://{username}:{password}@{server}/{database}'

# Create the SQLAlchemy engine to connect to the database
engine = create_engine(connection_string)

# Function to load data into MySQL
def load_data_to_mysql():
    try:
        # Load the data into MySQL tables (replace existing tables)
        PurchaseOrderDetail.to_sql('PurchaseOrderDetail', con=engine, if_exists='replace', index=False)
        Customers.to_sql('Customers', con=engine, if_exists='replace', index=False)
        SalesOrderDetail.to_sql('SalesOrderDetail', con=engine, if_exists='replace', index=False)
        SalesTerritory.to_sql('SalesTerritory', con=engine, if_exists='replace', index=False)
        ShipMethod.to_sql('ShipMethod', con=engine, if_exists='replace', index=False)
        Employee.to_sql('Employee', con=engine, if_exists='replace', index=False)

        print("Data loaded successfully into MySQL.")
    
    except Exception as e:
        print(f"Error loading data: {e}")


Data loaded successfully into MySQL.
Error checking tables: Not an executable object: 'SHOW TABLES'
